In [ ]:
%%time
%%capture
!pip install pydub > /dev/null

In [5]:
%%time
import random
import base64
import json
import tarfile
import wave
from contextlib import closing
from os import listdir, makedirs
from os.path import isfile, join
from pickle import dump
from sagemaker.mxnet import MXNet
from shutil import rmtree, copy2
from urllib.request import urlretrieve
from tempfile import gettempdir
import os
import boto3
import cv2
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt
import mxnet as mx
import numpy as np
import pandas as pd
import sagemaker
from pydub import AudioSegment

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 88 µs


In [ ]:
import shutil
shutil.rmtree('/tmp/ag-voice/pickles/')

In [ ]:
def get_labels(bucket_name):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    result = bucket.meta.client.list_objects(Bucket=bucket.name,
                                             Delimiter='/', Prefix='mp3/')
    labels = []
    for o in result.get('CommonPrefixes'):
        labels.append(o.get('Prefix').split("/")[1])
        label_indices = np.arange(0, len(labels))
    return labels, label_indices
labels, label_indices = get_labels('ag-voice')
print(labels)

In [ ]:
import os
os.listdir("/tmp/ag-voice/pickles")

In [2]:
labels = ["zero", "one", "two", "three", "four","five", "six", "seven", "eight", "nine"]

In [ ]:
import boto3
import os

def download_dir(client, resource, dist, label, local, bucket='ag-voice'):
    data = []
    train, validation, test = [], [], []
    prefix = dist + label
    conn = boto3.client('s3')
    for key in conn.list_objects(Bucket=bucket, Prefix= prefix)['Contents']:
        data.append( key['Key'])
#     random.shuffle(data)
    train = data[:int(round(len(data) * 0.8))]
    test = data[int(round(len(data) * 0.8)) : int(round(len(data) * 0.9))]
    validation = data[int(round(len(data) * 0.9)):]
    train_path = local + 'train'
    test_path = local + 'test'
    validation_path = local + 'validation'
    for key in train:
        file_name = key.split('mp3/')[1]
        if not os.path.exists(os.path.dirname(train_path + os.sep + file_name)): 
            os.makedirs(os.path.dirname(train_path + os.sep + file_name))
        resource.meta.client.download_file(bucket, key, train_path + os.sep + file_name)
    for key in test:
        file_name = key.split('mp3/')[1]
        if not os.path.exists(os.path.dirname(test_path + os.sep + file_name)): 
            os.makedirs(os.path.dirname(test_path + os.sep + file_name))
        resource.meta.client.download_file(bucket, key, test_path + os.sep + file_name)
    for key in validation:
        file_name = key.split('mp3/')[1]
        if not os.path.exists(os.path.dirname(validation_path + os.sep + file_name)): 
            os.makedirs(os.path.dirname(validation_path + os.sep + file_name))
        resource.meta.client.download_file(bucket, key, validation_path + os.sep + file_name)

client = boto3.client('s3')
resource = boto3.resource('s3')
labels = ["zero", "one", "two", "three", "four","five", "six", "seven", "eight", "nine"]
for label in labels:
    print(label)
    download_dir(client, resource, 'mp3/', label, '/tmp/ag-voice/mp3/')
    


In [ ]:
%%time
# rmtree("mxnet/wav", True)
# makedirs("/tmp/ag-voice/wav/")

sample_start = random.randint(500, 1000)
sample_finish = sample_start + 2000
for label in labels:
    print(label)
    path = "/tmp/ag-voice/mp3/train/" + label 
    wav_path = "/tmp/ag-voice/wav/train/" + label + "/"
    try:
        makedirs(wav_path)
    except:
        pass
    mp3_files = os.listdir(path)
    for mp3 in mp3_files:
        try: 
            sound = AudioSegment.from_mp3("/tmp/ag-voice/mp3/train/{}".format(label+"/"+mp3))
            sound.export("/tmp/ag-voice/wav/train/{}wav".format(label +"/"+ mp3[:-3]), format="wav")
        except:
            pass

In [51]:
%%time
# rmtree("mxnet/wav", True)
# makedirs("/tmp/ag-voice/wav/")

sample_start = random.randint(500, 1000)
sample_finish = sample_start + 2000
for label in labels:
    print(label)
    path = "/tmp/ag-voice/mp3/test/" + label 
    wav_path = "/tmp/ag-voice/wav/test/" + label + "/"
    try:
        makedirs(wav_path)
    except:
        pass
    mp3_files = os.listdir(path)
    for mp3 in mp3_files:
        try: 
            sound = AudioSegment.from_mp3("/tmp/ag-voice/mp3/test/{}".format(label+"/"+mp3))
            sound.export("/tmp/ag-voice/wav/test/{}wav".format(label +"/"+ mp3[:-3]), format="wav")
        except:
            pass

zero
one
two
three
four
five
six
seven
eight
nine
CPU times: user 0 ns, sys: 1min 7s, total: 1min 7s
Wall time: 2min 24s


In [3]:
def graph_spectrogram(wav_file, out):
    wav = wave.open(wav_file, "r")
    frames = wav.readframes(-1)
    sound_info = np.frombuffer(frames, "int16")
    frame_rate = wav.getframerate()
    wav.close()
    fig = plt.figure()
    fig.set_size_inches((1.4, 1.4))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.set_cmap("hot")
    plt.specgram(sound_info, Fs=frame_rate)
    plt.savefig(out, format="png")
    plt.close(fig)

In [6]:
%%time
%%capture --no-stdout --no-display
# rmtree("mxnet/spectrograms", True)
# makedirs("mxnet/spectrograms")

for label in labels:
    print(label)
    path = "/tmp/ag-voice/wav/train/" + label 
    spec_path = "/tmp/ag-voice/spectrograms/train/" + label + "/"
    try:
        makedirs(spec_path)
    except:
        pass
    wav_files = os.listdir(path)
    
    for wav in wav_files:
        graph_spectrogram("/tmp/ag-voice/wav/train/{}".format(label+"/"+wav), "/tmp/ag-voice/spectrograms/train/{}png".format(label+"/"+wav[:-3]))
        
        

zero
one
two
three
four
five
six
seven
eight
nine
CPU times: user 4min 16s, sys: 848 ms, total: 4min 17s
Wall time: 4min 17s


In [7]:
%%time
%%capture --no-stdout --no-display
# rmtree("mxnet/spectrograms", True)
# makedirs("mxnet/spectrograms")

for label in labels:
    path = "/tmp/ag-voice/wav/validation/" + label 
    spec_path = "/tmp/ag-voice/spectrograms/validation/" + label + "/"
    try:
        makedirs(spec_path)
    except:
        pass
    wav_files = os.listdir(path)
    
    for wav in wav_files:
        graph_spectrogram("/tmp/ag-voice/wav/validation/{}".format(label+"/"+wav), "/tmp/ag-voice/spectrograms/validation/{}png".format(label+"/"+wav[:-3]))
        
        

CPU times: user 32.5 s, sys: 136 ms, total: 32.7 s
Wall time: 32.7 s


In [8]:
def transform(row, label):
    img = cv2.imread(row)
    img = mx.nd.array(img)
    img = img.astype(np.float32)
    img = mx.nd.transpose(img, (2, 0, 1))
    img = img / 255
    target = np.float32(labels.index(label))
    return img, target

In [9]:
%%time

train_nd= []
for label in labels:
    path = "/tmp/ag-voice/spectrograms/train/" + label + "/"
    for spec_file in os.listdir(path):
        file = path + spec_file
        train_nd.append(transform(file, label))

CPU times: user 23.9 s, sys: 2.16 s, total: 26 s
Wall time: 10.8 s


In [10]:
%%time
validation_nd= []
for label in labels:
    path = "/tmp/ag-voice/spectrograms/validation/" + label + "/"
    for spec_file in os.listdir(path):
        file = path + spec_file
        validation_nd.append(transform(file, label))

CPU times: user 2.94 s, sys: 224 ms, total: 3.16 s
Wall time: 1.3 s


In [11]:
def save_to_disk(data, type):
    makedirs("{}/ag-voice/pickles/{}".format(gettempdir(), type))
    with open("{}/ag-voice/pickles/{}/data.p".format(gettempdir(), type), "wb") as out:
        dump(data, out)

In [12]:
%%time
rmtree("{}/ag-voice/pickles/".format(gettempdir()), True)

save_to_disk(train_nd, "train")
save_to_disk(validation_nd, "validation")

CPU times: user 2.2 s, sys: 3.5 s, total: 5.7 s
Wall time: 13 s


In [13]:
%%time
import sagemaker
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path="{}/ag-voice/pickles".format(gettempdir()),
                                       bucket="ag-voice", key_prefix="mxnet")
# rmtree("{}/pvdwgmas".format(gettempdir()), True)

CPU times: user 27 s, sys: 3.56 s, total: 30.6 s
Wall time: 21 s


In [20]:
estimator = MXNet("mxnet_script.py", 
          role=sagemaker.get_execution_role(), 
          train_instance_count=1, 
          train_instance_type="ml.p2.xlarge",
          hyperparameters={"epochs": 20},
          py_version="py3",
          framework_version="1.1.0")

In [21]:
estimator.fit(inputs)

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-304386803263
INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2018-09-11-22-55-45-243


................................
2018-09-11 23:00:51,739 INFO - root - running container entrypoint
2018-09-11 23:00:51,739 INFO - root - starting train task
2018-09-11 23:00:51,758 INFO - container_support.training - Training starting
2018-09-11 23:00:53,445 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'_scheduler_host': 'algo-1', 'hosts': ['algo-1'], 'base_dir': '/opt/ml', '_ps_port': 8000, 'user_script_name': 'mxnet_script.py', 'input_dir': '/opt/ml/input', 'code_dir': '/opt/ml/code', 'output_dir': '/opt/ml/output', 'available_cpus': 4, 'job_name': 'sagemaker-mxnet-2018-09-11-22-55-45-243', 'output_data_dir': '/opt/ml/output/data/', 'sagemaker_region': 'us-east-1', 'channel_dirs': {'training': '/opt/ml/input/data/training'}, 'input_config_dir': '/opt/ml/input/config', 'resource_config': {'hosts': ['algo-1'], 'network_interface_name': 'ethwe', 'current_host': 'algo-1'}, 'channels': {'training': {'TrainingInputMode': 'File', 'RecordWrapperType': 'None', 'S3DistributionTyp

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/sagemaker/session.py:861: DeprecationWarning: generator 'multi_stream_iter' raised StopIteration
  for idx, event in sagemaker.logs.multi_stream_iter(client, log_group, stream_names, positions):


Epoch 0. Loss: 2.0625021887, Train_acc 0.62703379224, Test_acc 0.584
Epoch 1. Loss: 0.838065698053, Train_acc 0.735794743429, Test_acc 0.683
Epoch 2. Loss: 0.697998545345, Train_acc 0.789612015019, Test_acc 0.749
Epoch 3. Loss: 0.612520291646, Train_acc 0.832916145181, Test_acc 0.779
Epoch 4. Loss: 0.556802900389, Train_acc 0.853566958698, Test_acc 0.784
Epoch 5. Loss: 0.493580023215, Train_acc 0.885106382979, Test_acc 0.795
Epoch 6. Loss: 0.447697070719, Train_acc 0.897997496871, Test_acc 0.806
Epoch 7. Loss: 0.410293374306, Train_acc 0.910888610763, Test_acc 0.808
Epoch 8. Loss: 0.366452069094, Train_acc 0.923404255319, Test_acc 0.819
Epoch 9. Loss: 0.343051449958, Train_acc 0.93266583229, Test_acc 0.82
Epoch 10. Loss: 0.312227013132, Train_acc 0.956195244055, Test_acc 0.83
Epoch 11. Loss: 0.275327359618, Train_acc 0.958573216521, Test_acc 0.839
Epoch 12. Loss: 0.248909196036, Train_acc 0.968961201502, Test_acc 0.829
Epoch 13. Loss: 0.228625586904, Train_acc 0.975844806008, Test_acc 

In [22]:
predictor = estimator.deploy(instance_type="ml.m4.xlarge", initial_instance_count=1)

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2018-09-11-22-55-45-243
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-2018-09-11-22-55-45-243


-----------------------------------------------------------------------------!

In [28]:
sagemaker_runtime_client = boto3.client("sagemaker-runtime")

In [64]:
with open("/tmp/ag-voice/wav/test/one/39a45230_nohash_2.wav", "rb") as audio_file:
    payload = base64.b64encode(audio_file.read()).decode("utf-8")
    response = sagemaker_runtime_client.invoke_endpoint(
        EndpointName=predictor.endpoint,
        Body=payload,
        ContentType="audio/wav",
        Accept="application/json"
    )["Body"].read()
    print("word 'one' predicted as {}".format(json.loads(response, encoding="utf-8")))

word 'one' predicted as one


In [ ]:
training_dir = "/tmp/ag-voice/pickles/"
from pickle import load
with open("{}/train/data.p".format(training_dir), "rb") as pickle:
    train_nd = load(pickle)
with open("{}/validation/data.p".format(training_dir), "rb") as pickle:
    validation_nd = load(pickle)

In [96]:
os.listdir('model/')

['bvlc_googlenet.tar.gz']